In [1]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
data = pd.read_parquet('data/concatenated_data.parquet')

In [5]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

C:\Users\Cliente\.conda\envs\xexeo\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
data['Tokenized Abstract'] = data['Abstract'].map(lambda x: tokenizer(x).input_ids)

645

In [30]:
tokenized_length = data['Tokenized Abstract'].map(len).values

max_length = int(np.percentile(tokenized_length, 99, dtype = int))
print(max_length)

data = data.loc[tokenized_length <= max_length]

561.0


In [41]:
input_ids = np.vstack(data['Tokenized Abstract'].map(lambda x: x + [0] * (max_length - len(x))).values)]
input_ids = torch.tensor(input_ids)

In [44]:
tokenized_titles = tokenizer(data['Title'].values.tolist(), return_tensors="pt", padding = True).input_ids

In [52]:
loss = model(input_ids=input_ids[:1000], labels=tokenized_titles).loss
loss.item()

RuntimeError: [enforce fail at C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 304248772608 bytes.

In [54]:
304248772608 / 1024 / 1024 / 1024

283.35375022888184